In [1]:
# instruciton datasetの生成

In [2]:
import os

#dataフォルダ内をリセット
os.system("mkdir data")
os.system("rm -rf data/*")

mkdir: cannot create directory ‘data’: File exists


0

In [3]:
from datasets import load_dataset
import json

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds_dict={}

# 自動生成したQAの読み込み

In [5]:
def clean_autogen(text):
    if text is None:
        return ""
    text=text.strip()
    return text

In [6]:
#original template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"
#custom template
#question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
#answer_template="<SEP>応答<SEP>"


#question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。:### 指示::"
#answer_template=":### 応答::"

In [7]:
#ベンチマークに用いるjmt benchと類似しすぎたデータは使わないようにする
import pandas as pd
#!pip install rapidfuzz
from rapidfuzz.process import cdist

jmt_bench_df=pd.read_csv("reference_data/jmtbench.csv")
bench_questions=jmt_bench_df["問い"].tolist()

def check_jmt_similarity(q,bench_questions):
    return 1  #一度チェックして問題ないなら、チェックしない
    scores = cdist([q], bench_questions,workers=1)
    score=max(scores[0])
    return score

# mixtralで自動生成したQ&A

In [8]:
from tqdm import tqdm
score_threshold=4
sim_threshold=80
records=[]

exclude_count=0


datasets=[
    load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train"),
    load_dataset("kanhatakeyama/OrcaJaMixtral8x22b",split="train"),

]
for dataset in datasets:
    for original_record in tqdm(iter(dataset)):
        q=clean_autogen(original_record["question"])
        a=clean_autogen(original_record["answer"])
        if q=="" or a=="":
            continue

        if "score" in original_record:
            if original_record["score"] is None:
                continue
            if int(original_record["score"])<score_threshold:
                continue

        if check_jmt_similarity(q,bench_questions)>sim_threshold:
            print("too similar to jmt bench",q)
            continue

        exclude_flag=False

        #回答しないパターンのrecordを除外
        for ng_word in ["申し訳","分からない","分かりません","すみません",
                        "図","表"]:
            if a.find(ng_word)>=0 or q.find(ng_word)>=0:
                #print("excluded:",a)
                exclude_flag=True
                exclude_count+=1
                continue

        if exclude_flag:
            continue
        #if len(a)<10:
        #    print("too short answer",a)
        #    continue

        text=f"{question_template}{q}{answer_template}{a}"
        if a!="":
            records.append(text)


ds_dict["auto_gen_mixtral"]=records
len(records),exclude_count

438565it [00:13, 31788.71it/s]
195895it [00:09, 21758.60it/s]


(349023, 135431)

# hachiさんのalpaca + mixtral dataset

In [9]:
hachi_alpaca=load_dataset("HachiML/Hachi-Alpaca",split='v1.0_cleaned')

In [10]:
records=[]
for record in tqdm(hachi_alpaca):
    q=record["instruction"]
    inp=record["input"]
    if inp!="":
        q+="\n"+inp
    a=record["output"]
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
    

ds_dict["hachi_alpaca"]=records

100%|██████████| 28861/28861 [00:02<00:00, 10553.69it/s]


# Bumpo dataset

In [11]:
#文法理解に関するデータセット
ds2=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
ds2

Dataset({
    features: ['instruction', 'question', 'answer'],
    num_rows: 26448
})

In [12]:
records=[]
for original_record in iter(ds2):
    q=(original_record["question"])
    a=(original_record["answer"])
    inst=(original_record["instruction"])
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
print(text)
ds_dict["bumpo_rikai"]=records
records[1]

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
問題　樹液からはチューインガムの原料が採取できる、中南米原産のアカテツ科の植物は何でしょう?

### 応答:
サポジラ


'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nfaカップ準決勝に延長戦はありますか\n\n### 応答:\nYES'

# minnade dataset

In [13]:
from datasets import load_dataset
#todo: dataを入れる
m_ds=load_dataset("minnade/chat-daily",split="train")

id_to_content={}
for record in m_ds:
    id_to_content[record["id"]]=record["body"]

questions=[]
for record in m_ds:
    if record["role"]=="assistant":
        q=id_to_content[record["parent_id"]]
        a=record["body"]
        #questions.append((q,a))
        text=f"{question_template}{q}{answer_template}{a}"
        questions.append(text)

ds_dict["minnade"]=questions

In [14]:
all_recrds=[]
for k,v in ds_dict.items():
    all_recrds+=v

len(all_recrds)

404649

In [15]:
import random

def write_jsonl(records,
    output_path="data/all.jsonl",
    n_eval=500,
    n_train=10**7,
    ):

    random.shuffle(records)
    df=pd.DataFrame()
    df["text"] =records[:-n_eval][:n_train]
    df["text"]=df["text"].astype(str)
    df=df.reset_index()
    df.to_parquet(output_path+".parquet")
    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    with open (output_path,"w", newline='\n') as f:
        temp_lines=lines[:-n_eval][:n_train]
        obj = map(lambda x: x + "\n", temp_lines)
        f.writelines(obj)


    df=pd.DataFrame()
    df["text"] =records[n_eval:]
    df["text"]=df["text"].astype(str)
    df=df.reset_index()
    df.to_parquet(output_path+"eval.parquet")
 
    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    lines=lines[-n_eval:]
    obj = map(lambda x: x + "\n", lines)
    with open (output_path+".eval","w", newline='\n') as f:
        f.writelines(obj)
    return df
n_train=10**7
n_train=10**5
df=write_jsonl(all_recrds,"data/all.jsonl",n_train=n_train)

# コード関連の抜き出し

In [70]:
def count_half_width_ratio(text):
    # 全文字数
    total_chars = len(text)
    # 半角文字数
    half_width_chars = sum(1 for char in text if ord(char) < 128)
    
    # 半角文字の割合を計算
    if total_chars == 0:
        return 0  # 文字列が空の場合は0を返す
    return half_width_chars / total_chars * 100
code_keywords=[
    "Python","python","code","コード","JSON","Java","XML","csv","CSV","def","list","html","HTML",
    "プログラム","スクリプト","script","Script"
]
code_records=[]

for record in all_recrds:
    for keyword in code_keywords:
        if record.find(keyword)>=0:
            if count_half_width_ratio(record)>12:
                code_records.append(record)
                break

len(code_records)


11166

In [73]:
for i in range(20):
    print(code_records[i].replace("\n","")[50:])

### 指示:この人物について、以下のを解答してください。1. 高校時代、ケーブルテレビで何をしていたか？2. 特技はなんですか？3. ジャズピアノが得意であること以外に、音楽関連でどのような能力や経験がありますか？4. 学位は何ですか？5. 吉崎典子とは何の関係でしたか？6. この人物が担当している業務について、「全力!脱力タイムズ」以外に、他になんて番組がありますか？### 応答:1. 高校時代、ケーブルテレビでは毎週金曜日のライブプログラム「ニューススタンバイ」を担当していました。2. 特技はジャズピアノとファッション。3. 音楽関連では、幼少時代からピアノを習っており、高校時代には学生時代のバンドでギターやベース、ドラムなども演奏することがありました。また、大学時代にはバンドを結成し、ライブ会場やカフェなどで活動していました。4. 学位は法学士（法学部卒業）です。5. 吉崎典子とは、東京大学法学部同窓生であり、友人関係です。6. この人物が担当している業務について、「全力!脱力タイムズ」以外には、NHKテレビの「ワールドビュー」やTBSテレビの「天才王子の逆襲!?〜僕たちには成功する方法がある〜」などがあります。
### 指示:複数の値を一度に編集したい場合や、簡略化した書き方を望む場合、まとめて記述することも可能です。以下のような形で、 shaardPrefereces.edit() 内に複数の値を設定できます。```javashaardPrefereces.edit()    .putInt("count1", count1)    .putInt("count2", count2)    .apply(); // or commit()```ただし、保存する値が少ない場合や、特に複数個を生成する必要がなければ、以前の方法でもありません。画面毎に分けたいといった場合は、複数個の SharedPreferences エディターを生成して使用することも可能です。```javashaardPrefereces1.edit()    .putInt("count", count)    .apply(); // or commit()shaardPrefereces2.edit()    .putString("name", name)    .apply();

# openmath instruct ja
